# Patient Safety SQL Agent Demo (SQLite Edition)

This notebook demonstrates a local SQL agent powered by Ollama (gemma3:4b) for querying a patient safety database using SQLite.

**Instructions:**
- No database setup or credentials needed!
- Ensure Ollama is running with the `gemma3:4b` model.
- Just run each cell in order.

In [1]:
# Install required packages
%pip install requests pandas faker tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. SQLite Database Setup
This will create a local SQLite database file called `patient_safety_demo.db`.

In [2]:
import sqlite3
DB_PATH = 'patient_safety_demo.db'

def get_connection():
    return sqlite3.connect(DB_PATH)

# Test connection
try:
    conn = get_connection()
    print('SQLite connection successful!')
    conn.close()
except Exception as e:
    print('SQLite connection failed:', e)

SQLite connection successful!


## 2. Create Database Schema
This will create all necessary tables in SQLite.

In [3]:
def create_schema():
    conn = get_connection()
    cursor = conn.cursor()
    # Drop tables if they exist (for demo repeatability)
    for table in ['Incident_Staff', 'Incidents', 'Admissions', 'Staff', 'Patients']:
        cursor.execute(f'DROP TABLE IF EXISTS {table}')
    # Create tables
    cursor.execute('''
        CREATE TABLE Patients (
            patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT,
            last_name TEXT,
            dob DATE,
            gender TEXT
        )
    ''')
    cursor.execute('''
        CREATE TABLE Staff (
            staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT,
            last_name TEXT,
            role TEXT,
            department TEXT
        )
    ''')
    cursor.execute('''
        CREATE TABLE Admissions (
            admission_id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            admission_date DATE,
            discharge_date DATE,
            department TEXT,
            attending_physician TEXT,
            FOREIGN KEY (patient_id) REFERENCES Patients(patient_id)
        )
    ''')
    cursor.execute('''
        CREATE TABLE Incidents (
            incident_id INTEGER PRIMARY KEY AUTOINCREMENT,
            admission_id INTEGER,
            incident_type TEXT,
            incident_date DATE,
            severity TEXT,
            description TEXT,
            reported_by INTEGER,
            FOREIGN KEY (admission_id) REFERENCES Admissions(admission_id),
            FOREIGN KEY (reported_by) REFERENCES Staff(staff_id)
        )
    ''')
    cursor.execute('''
        CREATE TABLE Incident_Staff (
            incident_id INTEGER,
            staff_id INTEGER,
            involvement_type TEXT,
            PRIMARY KEY (incident_id, staff_id, involvement_type),
            FOREIGN KEY (incident_id) REFERENCES Incidents(incident_id),
            FOREIGN KEY (staff_id) REFERENCES Staff(staff_id)
        )
    ''')
    conn.commit()
    print('Schema created.')
    cursor.close()
    conn.close()

create_schema()

Schema created.


## 3. Populate Database with Sample Data
This will generate and insert realistic demo data.

In [4]:
import random
from faker import Faker
from tqdm import tqdm

fake = Faker()

def populate_data():
    conn = get_connection()
    cursor = conn.cursor()
    # Patients
    patients = []
    for _ in range(60):
        gender = random.choice(['M', 'F', 'O'])
        dob = fake.date_of_birth(minimum_age=18, maximum_age=90)
        patients.append((
            fake.first_name_male() if gender=='M' else fake.first_name_female(),
            fake.last_name(),
            dob.isoformat(),  # Convert to string for SQLite
            gender
        ))
    cursor.executemany('INSERT INTO Patients (first_name, last_name, dob, gender) VALUES (?, ?, ?, ?)', patients)
    # Staff
    roles = ['Nurse', 'Physician', 'Technician', 'Pharmacist', 'Therapist']
    departments = ['ICU', 'ER', 'Surgery', 'Pediatrics', 'Oncology', 'Cardiology']
    staff = []
    for _ in range(40):
        staff.append((fake.first_name(), fake.last_name(), random.choice(roles), random.choice(departments)))
    cursor.executemany('INSERT INTO Staff (first_name, last_name, role, department) VALUES (?, ?, ?, ?)', staff)
    # Admissions
    admissions = []
    for pid in range(1, 61):
        for _ in range(random.randint(1, 3)):
            admission_date = fake.date_between(start_date='-3y', end_date='today')
            discharge_date = fake.date_between(start_date=admission_date, end_date='+30d')
            dept = random.choice(departments)
            attending = fake.name()
            admissions.append((
                pid,
                admission_date.isoformat(),
                discharge_date.isoformat(),
                dept,
                attending
            ))
    cursor.executemany('INSERT INTO Admissions (patient_id, admission_date, discharge_date, department, attending_physician) VALUES (?, ?, ?, ?, ?)', admissions)
    # Incidents
    incident_types = ['Fall', 'Medication Error', 'Infection', 'Pressure Ulcer', 'Equipment Failure', 'Other']
    severities = ['minor', 'moderate', 'severe', 'fatal']
    cursor.execute('SELECT admission_id FROM Admissions')
    admission_ids = [row[0] for row in cursor.fetchall()]
    cursor.execute('SELECT staff_id FROM Staff')
    staff_ids = [row[0] for row in cursor.fetchall()]
    incidents = []
    for _ in tqdm(range(200)):
        admission_id = random.choice(admission_ids)
        incident_type = random.choice(incident_types)
        incident_date = fake.date_between(start_date='-3y', end_date='today')
        severity = random.choices(severities, weights=[0.5, 0.3, 0.15, 0.05])[0]
        description = fake.sentence(nb_words=12)
        reported_by = random.choice(staff_ids)
        incidents.append((
            admission_id,
            incident_type,
            incident_date.isoformat(),
            severity,
            description,
            reported_by
        ))
    cursor.executemany('INSERT INTO Incidents (admission_id, incident_type, incident_date, severity, description, reported_by) VALUES (?, ?, ?, ?, ?, ?)', incidents)
    # Incident_Staff
    cursor.execute('SELECT incident_id FROM Incidents')
    incident_ids = [row[0] for row in cursor.fetchall()]
    incident_staff = []
    involvement_types = ['reporter', 'witness', 'responder']
    for iid in incident_ids:
        involved = random.sample(staff_ids, random.randint(1, 3))
        for sid in involved:
            involvement = random.choice(involvement_types)
            incident_staff.append((iid, sid, involvement))
    cursor.executemany('INSERT INTO Incident_Staff (incident_id, staff_id, involvement_type) VALUES (?, ?, ?)', incident_staff)
    conn.commit()
    print('Sample data populated.')
    cursor.close()
    conn.close()

populate_data()

100%|██████████| 200/200 [00:00<00:00, 30708.38it/s]

Sample data populated.


## 4. Ollama SQL Agent
This function sends a natural language prompt to Ollama and returns the generated SQL.

In [5]:
import requests

def ask_ollama(prompt, model='gemma3:4b'):
    url = 'http://localhost:11434/api/generate'
    payload = {
        'model': model,
        'prompt': prompt,
        'stream': False,
    }
    response = requests.post(url, json=payload)
    response.raise_for_status()
    return response.json()['response'].strip()

## 5. Natural Language to SQL Query Function
This function takes a natural language question, gets SQL from Ollama, runs it, and returns the results.

In [6]:
import pandas as pd

def nl_query_to_sql(nl_query):
    EXAMPLES = """
Example 1:
Q: Which patients were admitted to the ICU in 2024?
A: SELECT p.first_name, p.last_name, a.admission_date
   FROM Patients p
   JOIN Admissions a ON p.patient_id = a.patient_id
   WHERE a.department = 'ICU' AND a.admission_date LIKE '2024%';

Example 2:
Q: How many severe medication error incidents have been reported?
A: SELECT COUNT(*) 
   FROM Incidents
   WHERE incident_type = 'Medication Error' AND severity = 'severe';

Example 3:
Q: List the staff who have been involved in incidents as responders.
A: SELECT DISTINCT s.first_name, s.last_name, s.role, s.department
   FROM Staff s
   JOIN Incident_Staff isf ON s.staff_id = isf.staff_id
   WHERE isf.involvement_type = 'responder';

Example 4:
Q: What is the most common incident type in the Surgery department?
A: SELECT i.incident_type, COUNT(*) as count
   FROM Incidents i
   JOIN Admissions a ON i.admission_id = a.admission_id
   WHERE a.department = 'Surgery'
   GROUP BY i.incident_type
   ORDER BY count DESC
   LIMIT 1;

Example 5:
Q: Show me all admissions for Sarah Foster.
A: SELECT a.admission_date, a.discharge_date, a.department, a.attending_physician
   FROM Admissions a
   JOIN Patients p ON a.patient_id = p.patient_id
   WHERE p.first_name = 'Sarah' AND p.last_name = 'Foster';
"""

    prompt = (
        "Given the following SQLite schema: "
        "Patients(patient_id, first_name, last_name, dob, gender), "
        "Staff(staff_id, first_name, last_name, role, department), "
        "Admissions(admission_id, patient_id, admission_date, discharge_date, department, attending_physician), "
        "Incidents(incident_id, admission_id, incident_type, incident_date, severity, description, reported_by), "
        "Incident_Staff(incident_id, staff_id, involvement_type). "
        "Note: The department column is in the Admissions table, not in Incidents. "
        "To filter by department, you must join Incidents and Admissions on admission_id and use Admissions.department. "
        "All values in the severity column are lowercase. "
        "When joining tables with columns of the same name (like staff_id, patient_id), always prefix the column with the table name or alias (e.g., Staff.staff_id, Patients.patient_id). "
        "Here are some example questions and their SQL answers:\n"
        f"{EXAMPLES}\n"
        f"Convert this natural language request to a SQLite SQL query: {nl_query}"
    )
    sql_query = ask_ollama(prompt)
    # Remove code block markers if present
    sql_query = sql_query.strip()
    if sql_query.startswith('```'):
        sql_query = sql_query.split('\n', 1)[-1]
        sql_query = sql_query.rsplit('```', 1)[0]
    sql_query = sql_query.strip()
    print('Generated SQL:', sql_query)
    conn = get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(results, columns=columns)
    except Exception as e:
        print('Error running SQL:', e)
        return None
    finally:
        cursor.close()
        conn.close()

## 6. Example Queries
Try these or your own!

In [7]:
# Example: Show all severe incidents in the ICU
df = nl_query_to_sql('Show all moderate incidents in the ICU in 2022.')
df

Generated SQL: SELECT i.incident_id, i.incident_type, i.severity, i.description
FROM Incidents i
JOIN Admissions a ON i.admission_id = a.admission_id
WHERE a.department = 'ICU' AND a.admission_date LIKE '2022%' AND i.severity = 'moderate';


,incident_id,incident_type,severity,description
0,152,Medication Error,moderate,Worker growth near difference every live sudde...
1,155,Other,moderate,Attention everybody card network car phone int...


In [8]:
# Example: List patients who have had more than one incident
df = nl_query_to_sql('List patients who have had more than one incident and include a count of their incidents.')
df

Generated SQL: SELECT p.first_name, p.last_name, COUNT(i.incident_id) AS num_incidents
FROM Patients p
JOIN Admissions a ON p.patient_id = a.patient_id
JOIN Incidents i ON a.admission_id = i.admission_id
GROUP BY p.patient_id
HAVING COUNT(i.incident_id) > 1;


,first_name,last_name,num_incidents
0,Thomas,Webster,5
1,Erin,Burke,7
2,Crystal,Kelly,4
3,Grace,Jenkins,3
4,Christina,Daniel,4
5,Jeffrey,Ross,4
6,Daniel,Clark,3
7,Grant,Orr,2
8,Brandy,Ortiz,3
9,Jeffrey,Garcia,11


In [9]:
# Example: Which staff members have reported the most incidents?
df = nl_query_to_sql('Which members of the staff have reported the most incidents? Show me top five.')
df

Generated SQL: SELECT s.first_name, s.last_name, COUNT(i.incident_id) AS incident_count
FROM Staff s
JOIN Incident_Staff isf ON s.staff_id = isf.staff_id
JOIN Incidents i ON isf.incident_id = i.incident_id
GROUP BY s.staff_id
ORDER BY incident_count DESC
LIMIT 5;


,first_name,last_name,incident_count
0,Sharon,Jones,17
1,Joy,Hall,16
2,Amanda,Ryan,16
3,John,Norman,15
4,Abigail,Goodwin,14
